Full solution for Homework 1 number 5.0

In [1]:
import requests
import pandas as pd
import datetime as dt
import numpy as np
from pathlib import Path

# load the coasters
coaster = pd.read_csv(Path('data').joinpath('coaster_db.csv'))

# api url
api = 'https://earthquake.usgs.gov/fdsnws/event/1/query'

num_days = 90

#break up the range into 30 day intervalls
endtime = dt.date.today() - dt.timedelta(days = 1)

date_range = pd.date_range(start=endtime - dt.timedelta(days = num_days),
                           end=endtime, 
                           freq='30D')

eq_df = pd.DataFrame()

# get the data
for index in range(len(date_range[:-1])):
    
    # Create the payload for the GET request
    payload = {
        'format': 'geojson',
        'starttime': date_range[index],
        'endtime': date_range[index+1]
    }

    # get the data
    response = requests.get(api,
                            params = payload)

    # concatinate the data
    if response.status_code == 200:
        quake_json = response.json()
        eq_df = pd.concat([eq_df,
                           pd.DataFrame((quake['geometry']["coordinates"] for quake in quake_json['features'])
                                        , columns=['lat', 'long', 'depth'])])


# calculate distances
dist = []

for index, row in coaster.iterrows():
    dist_lat = eq_df['lat'] - row['latitude']
    dist_long = eq_df['long'] - row['longitude']
    dist.append(np.min(np.sqrt(dist_lat **2 + dist_long**2)))

coaster['eq_dist'] = dist

coaster.sort_values('eq_dist').head(1)

,coaster_name,Length,Speed,Location,Status,Opening date,Type,Manufacturer,Height restriction,Model,...,speed2,speed1_value,speed1_unit,speed_mph,height_value,height_unit,height_ft,Inversions_clean,Gforce_clean,eq_dist
280,Olympia Looping,"4,101 ft (1,250 m)",52 mph (84 km/h),Oktoberfest,Operating,"September 17, 1989",Steel,BHS,130 cm (4 ft 3 in),NaN,...,84 km/h,52.0,mph,52.0,110.0,ft,NaN,5,5.2,0.754652
